In [33]:
import pickle
import re
import pandas as pd

In [34]:
ori_text = "warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !"

In [35]:

import pandas as pd
import text_preprocessing as tp
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

#load train dataset
df_train = pd.read_csv('train_preprocess.tsv', sep='\t',header = None)
#df_train = pd.read_csv('/content/train_preprocess.tsv', sep='\t', header=None)
df_data = df_train.rename(columns={0: 'Text',1: 'Sentimen'})

#load kamus alay dataset
kamus_alay = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
#kamus_alay = pd.read_csv('/content/new_kamusalay.csv', encoding='latin-1', header=None)
kamus_alay = kamus_alay.rename(columns={0: 'original', 1: 'replacement'})

id_stopword_dict = pd.read_csv('stopwordbahasa.csv', header=None)
#id_stopword_dict = pd.read_csv('/content/stopwordbahasa.csv', header=None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})


# Define the preprocessing function
#text cleansing with regex
def lowercase(text):
    return text.lower()

def hapus_karakter_ga_penting(text):
    text = re.sub('\n',' ',text) # Hapus enter
    text = re.sub('nya|deh|sih',' ',text) # Hapus stopwords tambahan
    text = re.sub('RT',' ',text) # Hapus RT
    text = re.sub('USER',' ',text) # Hapus USER
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', ' ',text) # Hapus URL
    text = re.sub('  +', ' ', text) # Hapus extra spaces
    text = re.sub('[^a-zA-Z0-9]', ' ', text) #Hapus non huruf dan angka  
    text = re.sub('\@[a-zA-Z0-9]*', ' ', text) #Hapus non huruf atau apostrophe
    text = ' '.join([w for w in text.split() if len(w)>1]) #Hapus huruf tunggal 
    return text
    
def hapus_nonhurufangka(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

df_data_map = dict(zip(df_data['Text'], df_data['Sentimen']))
def normalisasi_alay(text):
    return ' '.join([df_data_map[word] if word in df_data_map else word for word in text.split(' ')])

def hapus_stopword(text):
    text = ' '.join(['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Hapus extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

def cleansing(text):
    text = lowercase(text) 
    text = hapus_nonhurufangka(text) 
    text = hapus_karakter_ga_penting(text) 
    text = normalisasi_alay(text) 
    text = stemming(text) 
    text = hapus_stopword(text) 
    return text

In [36]:
count_vect = pickle.load(open("feature_New3.sav", "rb"))
model_NN = pickle.load(open("model_NN3.sav", "rb"))

In [37]:
text = count_vect.transform([cleansing(ori_text)])

In [38]:
result = model_NN.predict(text)[0]
print("Sentiment:", result)

Sentiment: positive


# Prediksi menggunakan Package

In [39]:
import pickle
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
import text_preprocessing

# Download the Indonesian stopwords if not already downloaded

contoh = "warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !"
preprocessed_text = cleansing(contoh)
text_vector = count_vect.transform([preprocessed_text])

result = model_NN.predict(text_vector)[0]
print("text : ", contoh)
print("\ntext_new : ", preprocessed_text)
print("\nSentiment : ", result)



text :  warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !

text_new :  warung milik usaha pabrik puluh kenal putih bandung positive padu ahli masak padu kretivitas warung me jikan menu utama bahan menu ayam selera indonesia harga jangkau bletoka kalah asli tegal

Sentiment :  positive
